In [ ]:
#pragma cling add_include_path("/p/project/training2312/local/include")
#include <iostream>
#include <vector>
#include <algorithm>
#include <boost/type_index.hpp>
#include "Vbose.hh"
template <class T>
void typeof(T&& t) {
    std::cout << "Type name: " << boost::typeindex::type_id_with_cvr<T>().pretty_name() << "\n";
}

# Lambda functions: captures

## When

Variables which can be used inside lambda functions are those declared inside the lambda function, its formal arguments, global variables and variables captured by the capture brackets. Variables captured with `=` captures are copied into the lambda function. When does the copy take place ? Immediately when the lambda is declared or at the point when it is used ? How often does the copy happen, if we declare it once, but use it in a loop two million times ? Does `[=]` capture copy every variable defined in the context of the lambda function in to the lambda ? We will use the `Vbose` class to explore...

In [ ]:
{
    Vbose locvar{ "dinosaur" }, anothervar{"fish"};
    std::cout << "Declaring lambda function {\n";
    auto lambda = [=](int i) {
        std::cout << "\nIn lambda function, captured value of locvar is : "
                  << locvar.getval() << "\n";
        return i * i * i;
    };
    std::cout << "Declaring lambda function }\n";

    locvar.value("bird");
    for (int i = 0; i < 5; ++i) {
        std::cout << "Calling lambda function {\n";
        std::cout << 5 << " -> " << lambda(5) << "\n";
        std::cout << "Calling lambda function }\n";        
    }
}


How often did the copy operation for the captured variables occur inside the loop ? How did the change of the variable `locvar` after the declaration of the lambda reflect itself inside the lambda function, when it was called in the loop ? How do these results change if you use reference capture rather than value capture ?

In the following, we examine the effects of the `mutable` keyword for lambdas. The callable objects created when the lambda expression is evaluated are by default `const`. Imagine that the compiler is automatically generating a class with an overloaded function call operator, `operator()`. The captured variables are the arguments given to the constructor. By default, the `operator()` is `const` qualified, so that the callable object may not change state, i.e., the internal copies of the captured variables it creates in its constructor can not change when the lambda is used. However, declaring the lambda as `mutable` creates a non-constant function call operator. In this case, if any of the captured values are changed inside the lambda function call, those changes survive from call to call.

Mutable lambdas are also often used in connection with init-captures. We declare a new variable in the capture bracket, using an expression to initialize it. For instance `[i = 0UL]()mutable {}` makes a variable `i` available inside the lambda, but that variable is not something that was present in the surrounding scope, but was created along with the lambda, in its constructor. It is as if the variable was being declared with an `auto i = 0UL`. We can not drop the `auto` for variable declarations anywhere in C++, except in lambda capture brackets. This is because the capture brackets define all the parameters to be given to the constructor of the lambda: so it is a context where there can only be declarations. It was therefore formulated in this way.

In [ ]:
{
    std::string S{"Some string"};
    auto L = [S=1UL]()mutable { std::cout << "Inside Lambda, S = " << S << "\n"; ++S; };
    std::cout << "Outside Lambda, S= " << S << "\n";
    L();
    std::cout << "Outside Lambda, S= " << S << "\n";
    L();
    // You can understand the output by noting that despite the name S appearing in the capture
    // bracket, the S was "init captured" from an unsigned long. It has no connection whatsoever
    // with the variable S outside.
}

A popular application for mutable lambdas is generator functions. The algorithm `std::generate` takes the bounds of a sequence and a callable object of no input parameters (expected to be called just as `func()` with no inputs). A similar algorithm, `std::generate_n` accepts the start of the sequence, the number of elements it contains and a callable object. The generate and assign values for the sequence elements by calling the callable object once for each of them. Here is an example. We initialize a vector to the squares of positive integers. 

In [ ]:
{
    using namespace std;
    vector<unsigned long> v, w;
    generate_n(back_inserter(v), 10, [i = 0UL]() mutable {
        ++i;
        return i * i;
    });
    // v = [1, 4, 9, 16 ... ]
    std::cout << " v = \n";
    for (auto el : v)
        std::cout << el << "\n";
}

And in the following we do the same, but initialize a vector with the first 50 Fibonacci numbers...

In [ ]:
{
    using namespace std;
    vector<unsigned long> v, w;
    generate_n(back_inserter(w), 50, [ i = 0UL, j = 1UL ]() mutable {
        i = std::exchange(j, j + i);
        return i;
    });
    // w = [1, 1, 2, 3, 5, 8, 11 ...]

    std::cout << " w = \n";
    for (auto el : w)
        std::cout << el << "\n";
}